# Capstone project
## Assignment B1 Database to cloud
### Edgardo Alvarez

### MANAGING THE DATABASE ACCESS

Download the pymysql library to manage the mysql database

In [1]:
!pip install pymysql

Importing the needed libraries

In [2]:
import os
import sqlalchemy
SQLALCHEMY_SILENCE_UBER_WARNING = 1
SQLALCHEMY_WARN_20 = 0
import pandas as pd
import datetime
import seaborn as sns

Give value to all the variables to access to mysql database

In [3]:
db_host = '34.175.175.68'  # e.g. '127.0.0.1' ('172.17.0.1' if deployed to GAE Flex)
db_user = 'root' # e.g. 'my-db-user'
db_pass = 'MCSbt2023'  # e.g. 'my-db-password'
db_name = 'HyM' # e.g. 'my-database'
db_port = 3306  # e.g. 3306

Declare the function to connect to database

In [4]:
def connect_tcp_socket(
    db_host, db_user, db_pass, db_name, db_port = 3306
    ) -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of MySQL. """

    engine = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
    )
    return engine

Connect to the database via the above function

In [5]:
engine = connect_tcp_socket(db_host, db_user, db_pass, db_name, db_port)
conn = engine.connect()

In [6]:
print(conn)

Show tables already inserted in the DATABASE

In [7]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)
if not result:
  print("No tables in the DATABASE")

('KPI',)
('KPI_age',)
('KPI_date',)
('KPI_product_group',)
('KPI_saleschannel',)
('articles',)
('customers',)
('transactions',)


### LOADING THE DATA

Importing the needed libraries

Creating a function to remove all the unnamed columns in the datasets.

In [8]:
def remove_unnamed_cols(df):
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  return df

Creating a function to load the data directly from GOOGLE DRIVE.

In [10]:
def load_csv_from_drive(drive_url):
    url='https://drive.google.com/uc?id=' + drive_url.split('/')[-2]
    df = remove_unnamed_cols(pd.read_csv(url))
    return df

Loading the Articles data to pandas dataframe.

In [11]:
articles_drive_url = 'https://drive.google.com/file/d/1N9lK6lv_AgMIGvX3jFHcHLAWjRgIA9iF/view?usp=share_link'
articles_df = load_csv_from_drive(articles_drive_url)

Showing a sample of data to validate the columns.

In [12]:
articles_df.head(4)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


Applying **info** function to the dataframe to show types of the columns and the NULL values.

In [38]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

Detected that Detail_desc contains null values as it has less count than the others.

In [13]:
articles_df.fillna(value='no_detail', inplace=True)

Double checking that null values were replaced. 

In [14]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

Uploading data from dataframe to database

In [50]:
articles_df.to_sql(
    con = conn, name = "articles", if_exists = "replace")

105542

Verifying that the new table is created

In [51]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)
if not result:
  print("No tables in the DATABASE")

('articles',)
('customer',)


Loading the Customers data to dataframe.

Because the CSV file is too big to be pulled from google drive it has to be downloaded previously from the following URL: https://drive.google.com/file/d/16s7veu_B0B2PpFkR9JGQssNEu93FcFEB/view?usp=share_link

It also has to be stored in the same folder as this file to be pulled properly.

Unlike Articles database, Customer database is separated by ;, we need to specify this in order for pandas to separate the DF properly. 

In [15]:
customer_df=remove_unnamed_cols(pd.read_csv('customers.csv',sep=';'))

Showing a sample of data to validate the columns.

In [16]:
customer_df.head(2)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...


Applying **info** function to the dataframe to show types of the columns and the NULL values.

In [17]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1048575 non-null  object 
 1   FN                      364896 non-null   float64
 2   Active                  355236 non-null   float64
 3   club_member_status      1043971 non-null  object 
 4   fashion_news_frequency  1036271 non-null  object 
 5   age                     1036455 non-null  float64
 6   postal_code             1048575 non-null  object 
dtypes: float64(3), object(4)
memory usage: 56.0+ MB


In [18]:
customerclean_df=customer_df.dropna(inplace=False)

In [19]:
customerclean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354072 entries, 4 to 1048568
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             354072 non-null  object 
 1   FN                      354072 non-null  float64
 2   Active                  354072 non-null  float64
 3   club_member_status      354072 non-null  object 
 4   fashion_news_frequency  354072 non-null  object 
 5   age                     354072 non-null  float64
 6   postal_code             354072 non-null  object 
dtypes: float64(3), object(4)
memory usage: 21.6+ MB


Changed age to Integer

In [20]:
customerclean_df=customerclean_df.astype({'age':'int'})

In [21]:
customerclean_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
13,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,1.0,1.0,ACTIVE,Regularly,56,b31984b20a8c478de38eaf113c581ff64e63c4242e607b...
14,0000ae1bbb25e04bdc7e35f718e852adfb3fbb72ef38b3...,1.0,1.0,ACTIVE,Regularly,29,2c29ae653a9282cce4151bd87643c907644e09541abc28...
15,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,1.0,1.0,ACTIVE,Regularly,54,ca8ca81e8b5794992144273b0eada83a7e09ec728c1093...


Uploading to SQL

In [113]:
customerclean_df.to_sql(
    con = conn, name = "customers", if_exists = "replace")



In [8]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)
if not result:
  print("No tables in the DATABASE")


('articles',)
('customers',)


In [19]:

transactions_drive_url = 'https://drive.google.com/file/d/1lxbPMnJkqLZ9VJg38e7LydcL27XodfnU/view?usp=share_link'
transactions_df = load_csv_from_drive(transactions_drive_url)

In [22]:
transactions_df=remove_unnamed_cols(pd.read_csv('transactions.csv'))

In [23]:
transactions_df.head(4)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2


In [9]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


Changing t_dat to datetime dtype and changing column name to date

In [24]:
transactions_df['t_dat']= pd.to_datetime(transactions_df['t_dat'])
transactions_df.rename(columns = {'t_dat':'date'}, inplace = True)  

In [25]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   customer_id       object        
 2   article_id        int64         
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.2+ GB


In [26]:
transactions_df.head(4)

,date,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2


In [13]:
transactions_df.to_sql(
    con = conn, name = "transactions", if_exists = "replace")

31788324

In [35]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)
if not result:
  print("No tables in the DATABASE")

('KPI',)
('KPI_age',)
('KPI_date',)
('KPI_product_group',)
('KPI_saleschannel',)
('articles',)
('customers',)
('transactions',)


In [27]:
transactionsnarticles_df = pd.merge(transactions_df, articles_df, on='article_id', how='left')

In [28]:
transactionsnarticles_df.head(4)    

,date,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,663713,Atlanta Push Body Harlow,283,Underwear body,Underwear,...,Expressive Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up body with underwired, moulded, pa..."
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,541518,Rae Push (Melbourne) 2p,306,Bra,Underwear,...,Casual Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up bras with underwired, moulded, pa..."
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,505221,Inca Jumper,252,Sweater,Garment Upper body,...,Tops Knitwear DS,D,Divided,2,Divided,58,Divided Selected,1003,Knitwear,Jumper in rib-knit cotton with hard-worn detai...
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,685687,W YODA KNIT OL OFFER,252,Sweater,Garment Upper body,...,Campaigns,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1023,Special Offers,V-neck knitted jumper with long sleeves and ri...


In [41]:
transactionsnarticles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 0 to 31788323
Data columns (total 29 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   date                          datetime64[ns]
 1   customer_id                   object        
 2   article_id                    int64         
 3   price                         float64       
 4   sales_channel_id              int64         
 5   product_code                  int64         
 6   prod_name                     object        
 7   product_type_no               int64         
 8   product_type_name             object        
 9   product_group_name            object        
 10  graphical_appearance_no       int64         
 11  graphical_appearance_name     object        
 12  colour_group_code             int64         
 13  colour_group_name             object        
 14  perceived_colour_value_id     int64         
 15  perceived_colour_value_name   

In [29]:
KPI_df = transactionsnarticles_df[['date', 'customer_id', 'price', 'sales_channel_id', 'product_group_name', 'section_name']].copy()

In [30]:
KPI_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   customer_id         object        
 2   price               float64       
 3   sales_channel_id    int64         
 4   product_group_name  object        
 5   section_name        object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.7+ GB


In [31]:
KPI_df = pd.merge(KPI_df, customerclean_df[['customer_id', 'age', 'club_member_status']], on='customer_id')

In [32]:
KPI_df.head(4)

,date,customer_id,price,sales_channel_id,product_group_name,section_name,age,club_member_status
0,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.015237,2,Garment Upper body,Divided Selected,32,ACTIVE
1,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.016932,2,Garment Upper body,Womens Everyday Collection,32,ACTIVE
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.016932,2,Garment Upper body,Womens Everyday Collection,32,ACTIVE
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.016932,2,Garment Upper body,Womens Everyday Collection,32,ACTIVE


In [56]:
KPI_date = KPI_df.groupby(KPI_df['date'].dt.day_name())['price'].agg([('revenue', 'sum')]).reset_index()
KPI_date['revenue'] = KPI_date['revenue'].round(2)

In [57]:
print(KPI_date)


        date   revenue
0     Friday  41825.01
1     Monday  36012.96
2   Saturday  43881.20
3     Sunday  36318.46
4   Thursday  43812.56
5    Tuesday  37138.85
6  Wednesday  43215.68


In [58]:
KPI_age = KPI_df.groupby('age')['price'].agg([('revenue', 'sum')]).reset_index()
KPI_age['revenue'] = KPI_age['revenue'].round(2)
print(KPI_age)

     age  revenue
0   16.0    11.91
1   17.0   391.31
2   18.0  1470.29
3   19.0  3926.14
4   20.0  7083.21
..   ...      ...
76  94.0     0.99
77  95.0     1.28
78  97.0     0.76
79  98.0     0.27
80  99.0     1.60

[81 rows x 2 columns]


In [59]:
KPI_df['sales_channel_id'] = KPI_df['sales_channel_id'].map({1: 'online', 2: 'offline'})
KPI_saleschannel = KPI_df.groupby('sales_channel_id')['price'].agg([('revenue', 'sum')]).reset_index()
KPI_saleschannel['revenue'] = KPI_saleschannel['revenue'].round(2)
print(KPI_saleschannel)

  sales_channel_id    revenue
0          offline  211165.49
1           online   71039.22


In [1]:
KPI_product_group = KPI_df.groupby('product_group_name')['price'].agg([('revenue', 'sum')]).reset_index()
KPI_product_group['revenue'] = KPI_product_group['revenue'].round(2)
print(KPI_product_group)

NameError: name 'KPI_df' is not defined

In [33]:
KPI_df.to_sql(
    con = conn, name = "KPI", if_exists = "replace")

10228628

In [63]:
KPI_age.to_sql(
    con = conn, name = "KPI_age", if_exists = "replace")

81

In [64]:
KPI_date.to_sql(
    con = conn, name = "KPI_date", if_exists = "replace")

7

In [65]:

KPI_product_group.to_sql(
    con = conn, name = "KPI_product_group", if_exists = "replace")


19

In [66]:

KPI_saleschannel.to_sql(
    con = conn, name = "KPI_saleschannel", if_exists = "replace")


2